In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import itertools
import seaborn as sns

In [2]:
# import data
df = pd.read_csv('textclean5.csv')
df.head(5)

,Unnamed: 0,x
0,1,cc ya banget ptn pamor kayak u pts pamor kayak...
1,2,cc tragedi trisakti peristiwa tembak mei hadap...
2,3,cc huhu gue logo trisakti jeplak anjir suruh g...
3,4,cc berita bidang iii fh usakti seminar sosiali...
4,5,cc nder trisakti


In [3]:
# menghapus tweet duplikat

df = df.drop_duplicates()

In [4]:
len(df)

1748

In [5]:
len(df[df['x'].isnull()==True])

0

In [6]:
df = df.dropna(subset=['x'])

In [7]:
df.isnull().sum()

Unnamed: 0    0
x             0
dtype: int64

In [8]:
def count_words(x):
    words = word_tokenize(x)
    n=len(words)
    return n

In [9]:
#menghitung word length
df['word_length'] = df['x'].apply(lambda x:count_words(x))

In [10]:
df['word_length'].value_counts().sort_index()

1        15
2        65
3       171
4       216
5       189
6       151
7       130
8       122
9       128
10      114
11      100
12       89
13       96
14       76
15       49
16       14
17       16
18        2
19        2
20        2
2657      1
Name: word_length, dtype: int64

In [11]:
#word length 0 di drop
df = df.drop(df[df['word_length']==0].index,axis=0)
df = df.reset_index(drop=True)

In [12]:
#membuat kamus kata
#meghitung berapa banyak kata yang ada di dataset

word_dict = {}
for i in range(0,len(df['x'])):
    sentence = df['x'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in word_dict:
            word_dict[j] = 1
        else:
            word_dict[j] += 1

In [13]:
len(word_dict)

3985

In [14]:
#import lexicon dan menghapus kata-kata negasi dari lexicon
negasi = ['bukan','tidak','ga','gk']
lexicon = pd.read_csv('modified_full_lexicon.csv')
lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')
                              |(lexicon['word'] == 'tidak')
                              |(lexicon['word'] == 'ga')|(lexicon['word'] == 'gk') ].index,axis=0)
lexicon = lexicon.reset_index(drop=True)

In [15]:
len(lexicon)

10248

In [16]:
lexicon.head(10)

,word,weight,number_of_words
0,hai,3,1
1,merekam,2,1
2,ekstensif,3,1
3,paripurna,1,1
4,detail,2,1
5,pernik,3,1
6,belas,2,1
7,welas,4,1
8,kabung,1,1
9,rahayu,4,1


In [17]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']

In [18]:
len(lexicon_word)

10248

In [19]:
#memeriksa apakah ada kata dalam kamus yang tidak termasuk dalam lexicon
ns_words = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words.append(word)
len(ns_words)

3151

In [20]:
lexicon['number_of_words'].value_counts()

1    9536
2     686
3      24
4       2
Name: number_of_words, dtype: int64

In [21]:
'covid' in word_dict

True

In [22]:
'budi baik' in lexicon_word

True

In [24]:
#menghitung sentiment kata dengan menghitungnya mejadi lexicon
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []

#fungsi untuk menuliskan sentiment kata jika sudah ditemukan
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # jika sudah termasuk dalam bag of words matrix, maka tambahkan nilainya
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #jika tidak tambahkan kata baru
        sencol.append(word)
        sen.append(1)
        add += 1
    #jika ada kata negasi sebelumnya, setiment adalah negasi sentimentnya
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# memeriksa setiap kata, jika muncul dalam lexicon, dan menghitung sentimennya
for i in range(len(df)):
    nsen = senrow.shape[0]
    words = word_tokenize(df['x'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # periksa apakah kata termasuk dalam lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # jika tidak cek kata dasarnya
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # jika masih negatif, coba cocokan kombinasi kata dengan kata yang berdekatan
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # jika ada kata baru yang ditemukan, maka perluas matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    #jika tidak ada perbarui matriks lama
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

In [30]:
len(sentiment_list)

9995

In [31]:
print(senrow.shape[0])

9995


In [34]:
sencol.append('sentiment')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))
df_sen = pd.DataFrame(sentiment_data,columns = sencol)

In [35]:
df_sen

,yang,mengikuti,kegiatan,berkerumun,minta,usul,beli,buat,lancar,banding,...,bablas,abaikan,pinter,check,kredibel,pecat,ulah,pls,ngasih,sentiment
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
1,0,0,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,9
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
9991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
9992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1


In [36]:
# melihat sentimen dari tweet original

cek_df = pd.DataFrame([])
cek_df['tweet'] = df['tweet'].copy()
cek_df['sentiment']  = df_sen['sentiment'].copy()

In [37]:
cek_df.head(5)

,tweet,sentiment
0,Warga yang pernah mengikuti kegiatan berkerum...,-1
1,Kemenag Usul Beli Vaksin Covid-19 Buatan Saud...,9
2,RT @zamirmohyedin: Perbandingan antara vaksin...,3
3,Indonesia memiliki infrastruktur dan SDM mump...,1
4,"Perebutan vaksin Covid-19, mirip fenomena keb...",5


In [38]:
#cek_df.to_csv("vaksin_sentiment.csv")